In [ ]:
import tkinter
from tkinter import *

import nltk
from nltk.stem.lancaster import LancasterStemmer
import tensorflow
import tflearn
import random
import numpy as np
import pickle

stemmer=LancasterStemmer()
datos = []
listDict = []

import json
with open(r'C:\Users\rosi7\Downloads\hunterFINAL.json') as file:
    data = json.load(file)
    
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return np.array(bag)

base = Tk()
base.title("Hunter")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="grey86", height="8", width="50", font="Courier", wrap=WORD)
ChatLog.config(foreground="grey50", font=("Courier",12))
ChatLog.place(x=6, y=6, height=386, width=370)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand']=scrollbar.set
scrollbar.place(x=376, y=6, height=386)

ChatLog.config(state=DISABLED)

def chatbot_response(msg):
    
    words=[]
    labels=[]
    docs_x=[]
    docs_y=[]
    
    for intents in data["hunterFINAL"]:
        for patterns in intents['patterns']:
            wrds = nltk.word_tokenize(patterns)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intents["tag"])
            
            if intents['tag'] not in labels:
                labels.append(intents['tag'])
    
    words=[stemmer.stem(w.lower()) for w in words if w != "?"]
    
    words = sorted(list(set(words)))
    
    labels = sorted(labels)
    
    training=[]
    output=[]
    
    out_empty = [0 for _ in range (len(labels))]
    
    for x, doc in enumerate(docs_x):
        bag = []
        
        wrds= [stemmer.stem(w.lower()) for w in doc]
        
        for w in words:
            if w in wrds:
                
                bag.append(1)
            else:
                bag.append(0)
            
            output_row = out_empty[:]
            output_row[labels.index(docs_y[x])] = 1
            
            training.append(bag)
            output.append(output_row)
    
    training = np.array(training)
    output = np.array(output)
    
    with open ("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
        
        tensorflow.compat.v1.reset_default_graph()
        
        net = tflearn.input_data(shape=[None, len(training[0])])
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
        
        net = tflearn.regression(net)
        
        model = tflearn.DNN(net)
        
    try:
        model.load("model.tflearn")
    except:
        
        tensorflow.compat.v1.reset_default_graph()
        net = tflearn.input_data(shape=[None, len(training[0])])
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
        
        net = tflearn.regression(net)
        
        model = tflearn.DNN(net)
        
        model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
        model.save("model.tflearn")
        
    results = model.predict([bag_of_words(msg,words)])
    
    results_index = np.argmax(results)
    
    tag = labels[results_index]
    
    for tg in data["hunterFINAL"]:
        if tg["tag"] == tag:
            responses = tg['responses']
            
    return(random.choice(responses))

def send():
    global NomNum
    if NomNum == True:
        msg=EntryBox.get("1.0", "end-1c").strip()
        EntryBox.delete("0.0",END)
        res=chatbot_response(msg)
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You : "+msg+'\n\n')
        ChatLog.config(foreground="dodger blue", font=("Courier",12))
        ChatLog.insert(END, "ChatBOT : "+res+'\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
def getName():
    global ciclo
    global NomNum
    if NomNum == False:
        if ciclo == 0:
            dat = EntryBox.get("1.0", "end-1c").strip()
            datos.append(dat)
            print(datos)
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, " You : "+dat+ '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            EntryBox.delete("0.0",END)
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "Hunter: please insert your telephone number.\n\n")
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            ciclo = ciclo + 1
        
        elif ciclo ==1:
            dat = EntryBox.get("1.0", "end-1c").strip()
            datos.append(dat)
            print(datos)
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, " You : "+dat+ '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            EntryBox.delete("0.0",END)
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "Hunter: OK, received what can I help you with?.\n\n")
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            nombreDict = dict(zip(["Nombre","Numero"],datos))
            listDict.append(nombreDict)
            print(nombreDict)
            print(listDict)
            
            try:
                with open("clientData.json")as fp:
                    listObj = json.load(fp)
                listObj.append(nombreDict)
                with open("clientData.json", "w") as json_file:
                    json.dump(listObj, json_file, indent=4, separators=(",",": "))
            except:
                with open("clientData.json", "w") as outfile:
                    json.dump(listDict, outfile, indent=4)
            NomNum = True
            
            
            
SendButton = Button(base, font=("Courier", 12, 'bold'), text="Send", width=9,
                   height=5, bd=0, bg="dodger blue", activebackground="gold", 
                    fg='#ffffff', command=lambda:[send(),getName()])

SendButton.place(x=282, y=401, height=90)

EntryBox = Text(base, bd=0, bg="grey59", width="29", height="5", font="Courier")
EntryBox.place(x=6, y=401, height=90, width=265)

NomNum = False
ciclo = 0

ChatLog.config(state=NORMAL)
ChatLog.insert(END, "ChatBot: Hi, please insert your name and phone number.\n\n")
ChatLog.insert(END, "ChatBot: Please insert your name:\n\n")
ChatLog.config(state=DISABLED)
ChatLog.yview(END)


base.bind('<Return>', lambda event:[send(),getName()])
base.mainloop()         
            
           
            
                               

Training Step: 30292  | total loss: 0.00000 | time: 58.770s
| Adam | epoch: 060 | loss: 0.00000 - acc: 1.0000 -- iter: 3032/4050
